In [1]:
import sys,os
import numpy as np

import keras
import pandas as pd
import re
import copy #can delete

import string
import matplotlib.pyplot as plt


from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.preprocessing.text import Tokenizer
from keras import optimizers
from keras import regularizers

from nltk.corpus import stopwords
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import classification_report,accuracy_score,f1_score
from sklearn.cross_validation import cross_val_score,cross_val_predict,KFold,StratifiedKFold

Using Theano backend.
/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
np.random.seed(1)

MAX_SEQUENCE_LENGTH = 30 #max number of sentences in a message
MAX_NB_WORDS = 20000 #cap vocabulary
GLOVE_FILE = 'glove.twitter.27B/glove.twitter.27B.200d.txt'
EMBEDDING_DIM = 200 #size of word vector 
TWITTER_FILE = 'training-Obama-Romney-tweets.xlsx'
JAR_FILE = '/home/sreeraj/stanford-postagger-2016-10-31/stanford-postagger.jar'
MODEL_FILE = '/home/sreeraj/stanford-postagger-2016-10-31/models/english-left3words-distsim.tagger'
TOKENIZER = 'keras' #or use nltk
STEMMER = 'wordnet' #or use snowball or porter

In [3]:
def get_Ytrue_Ypred(model,x,y):
    #Y matrix is [1,0,0] for class 0, [0,1,0] for class 1, [0,0,1] for class -1
    convert_to_label ={0:0,1:1,2:-1}
    model_predictions = model.predict(x)
    y_pred = np.zeros(len(y))
    y_true = np.zeros(len(y))
    #errors = 0.0
    for i in range(len(y)):
        y_pred[i] = convert_to_label[np.argmax(model_predictions[i])]
        y_true[i] = convert_to_label[np.argmax(y[i])]
        #if y_true[i] != y_pred[i]:
            #errors+=1.0
    return y_true,y_pred
    

# read the data
obama_data = pd.read_excel(TWITTER_FILE,names = ['date','time','text','sentiment'],parse_cols = 4,sheetname = 'Obama')
romney_data = pd.read_excel(TWITTER_FILE,names = ['date','time','text','sentiment'],parse_cols = 4,sheetname = 'Romney')

In [4]:
def get_data(data):
    """ get and clean the data """
    data = data.iloc[1:]
    data['text'] = data['text'].values.astype('unicode')
    data['date'] = data['date'].values.astype('str')
    data['time'] = data['time'].values.astype('unicode')
    # remove rows with mixed sentiment
    data = data[data['sentiment'] < 2]
    data.index = range(len(data))
    
    return data

obama_data = get_data(obama_data)
romney_data = get_data(romney_data)

obama_dataO = copy.deepcopy(obama_data)

print obama_data.head()
print romney_data.head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

                  date            time  \
0  2012-10-16 00:00:00  10:28:53-05:00   
1  2012-10-16 00:00:00  10:04:30-05:00   
2  2012-10-16 00:00:00  09:50:08-05:00   
3  2012-10-16 00:00:00  10:00:16-05:00   
4  2012-10-16 00:00:00  09:48:07-05:00   

                                                text sentiment  
0  Kirkpatrick, who wore a baseball cap embroider...         0  
1  #<e>obama</e> debates that Cracker Ass Cracker...         1  
2  @Hollivan @hereistheanswer  Youre missing the ...         0  
3  I was raised as a Democrat  left the party yea...        -1  
4  The <e>Obama camp</e> can't afford to lower ex...         0  
                  date            time  \
0  2012-10-16 00:00:00  09:38:08-05:00   
1  2012-10-16 00:00:00  10:14:18-05:00   
2  2012-10-16 00:00:00  09:27:16-05:00   
3  2012-10-16 00:00:00  10:11:43-05:00   
4  2012-10-16 00:00:00  10:13:17-05:00   

                                                text sentiment  
0  Insidious!<e>Mitt Romney</e>'s Bain 

In [5]:
emoticon_dictionary = {':)':' smileyface ','(:':' smileyface ','XD': ' happyface ',':D': ' smileyface ','>.<':' smileyface ',':-)':' smileyface ',';)':' winkface ',';D':' winkface ',':\'(':' cryingface '}

emoticons = [':\)','\(:','XD',':D','>\.<',':-\)',';\)',';D',':\'\(']

emoticon_pattern = re.compile(r'(' + '\s*|\s*'.join(emoticons) + r')')

# convert emoticons to words
def emoticon_converter(x):
    x = emoticon_pattern.sub(lambda i : emoticon_dictionary[i.group().replace(' ','')],x)   
    return x

obama_data['text'] = obama_data['text'].apply(emoticon_converter)
romney_data['text'] = romney_data['text'].apply(emoticon_converter)

In [6]:
def separate_hashtag(x):
    x = x.split()
    temp = []
    for word in x:
        if '#' in word:
            temp += re.findall('[A-Z][^A-Z]*',word)
        else:
            temp.append(word)
    
    return ' '.join(temp)

obama_data['text'] = obama_data['text'].apply(separate_hashtag)
romney_data['text'] = romney_data['text'].apply(separate_hashtag)

In [7]:
# remove punctuations
punc = ['\:','\;','\?','\$','\.','\(','\)','\#','\=','\%','\-','\>','\<','\,']
cond_1 = re.compile('|'.join(punc))
# remove tags
tags = ['<a>','</a>','<e>','</e>']
cond_2 = re.compile("|".join(tags))

def preprocess(data):
    """ preprocess the data"""
     # remove users
    data = data.apply(lambda x : re.sub(r'\@\s?\w+','',x))
    # remove hypertext 
    data = data.apply(lambda x : re.sub(r'http://\S+','',x))
    # remove tags
    data = data.apply(lambda x : re.sub(cond_2,'',x))
    # remove punctuations
    data = data.apply(lambda x : re.sub(cond_1,'',x))
    # remove digits
    data = data.apply(lambda x : re.sub(r'[0-9]+','',x))
    # convert to ascii
    data = data.apply(lambda x: x.encode('utf-8'))
    
    return data

obama_data['text'] = preprocess(obama_data['text'])
romney_data['text'] = preprocess(romney_data['text'])

In [8]:
## IMP - Process Emoticons, better stopwords list, clean hashtags
# Tweet NLP

manual_stopwords_list = ['RT']

# stopwords list based on pos tags

from nltk.tag import StanfordPOSTagger

jar = JAR_FILE
model = MODEL_FILE

st = StanfordPOSTagger(model, jar, encoding='utf8')

remove_tags_pos = ['IN','DT','PRP','PRP$','WDT','WP','WP$','CD','PDT']
remove_tags_tweetnlp = []


def tweet_tag_filter(x):
    pass


# obama_data['text'] = obama_data['text'].apply(tweet_tag_filter)
# romney_data['text'] = romney_data['text'].apply(tweet_tag_filter)


def pos_tag_filter(x):
    x = x.split()
    s = st.tag(x)
    for i,(_,tag) in enumerate(s):
        if tag in remove_tags_pos:
            x[i] = ''
    return ''.join(x)
    

# obama_data['text'] = obama_data['text'].apply(pos_tag_filter)
# romney_data['text'] = romney_data['text'].apply(pos_tag_filter)

# remove stopwords


stopwords_list = stopwords.words('english') + manual_stopwords_list


In [9]:
# stemming
class Tokenizer(object):
    def __init__(self,stemmer='porter'):
        self.stemmer = stemmer
        if stemmer == 'wordnet':
            self.wnl = WordNetLemmatizer()
        if stemmer == 'porter':
            self.wnl = PorterStemmer()
        if stemmer == 'snowball':
            self.wnl = SnowballStemmer('english')
    def __call__(self, doc):
        if self.stemmer == 'wordnet':
            return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
        else:
            return [self.wnl.stem(t) for t in word_tokenize(doc)]

In [10]:
def get_X_y(data):
    return data['text'],data['sentiment'].astype(int)

texts = romney_data['text']
labels = np.array(romney_data['sentiment'])

In [11]:
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts) #list of lists, basically replaces each word with number

tokens = []
myTokenizer = Tokenizer(STEMMER)
for i in range(0,len(texts)):
    try:
        tokens.append(myTokenizer.__call__(texts[i]))
    except UnicodeDecodeError:
        pass
word_dict = {}
winx = 1
mysequences = []
tsq = []
for i in range(0,len(tokens)):
    for token in tokens[i]:
        if token not in word_dict:
            word_dict[token] = winx
            winx += 1
        tsq.append(word_dict[token])
    mysequences.append(tsq)
    tsq = []

In [12]:
word_index = tokenizer.word_index #key = word, value = number
#word_index = word_dict
#sequences = mysequences
if TOKENIZER == 'nltk':
    word_index = word_dict
    sequences = mysequences
print('Found %s unique tokens.' % len(word_index))

#pad the data 
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print labels[0:4]
Y = labels
labels = keras.utils.np_utils.to_categorical(labels,nb_classes=3)
print labels[0:4]

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 8662 unique tokens.
[-1 -1 -1 -1]
[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]
('Shape of data tensor:', (5648, 30))
('Shape of label tensor:', (5648, 3))


In [13]:
embeddings_index = {}
f = open(GLOVE_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

#prepare embedding matrix

#num_words = min(MAX_NB_WORDS, len(word_index))
num_words = len(word_index)+1
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 1193514 word vectors.


In [14]:
np.random.seed(1)

def build_model():
    np.random.seed(1)
    l2 = regularizers.l2(0.01)
    l22 = regularizers.l2(0.01)
    model = Sequential()
    embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=1)
    model.add(embedding_layer)
    #model.add(LSTM(10,return_sequences=True))
    model.add(LSTM(150,return_sequences=False,W_regularizer=l2))
    #model.add(LSTM(15,return_sequences=False,W_regularizer=l22))
    model.add(Dropout(0.2))
    #model.add(Dense(10, activation='relu'))
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.8, nesterov=True)
    model.add(Dense(len(labels[0]), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    return model

In [15]:
#k fold cross validaiton
avg_acc = []
avg_f1 = []

#data = data[0:500]
kf = KFold(n=len(data),n_folds=10)
#kf = StratifiedKFold(Y,n_folds=10)
for train,test in kf: #do the cross validation
    np.random.seed(1)
    x_train, x_val, y_train, y_val = data[train], data[test], labels[train], labels[test]
    
    model = build_model()
    model.fit(x_train, y_train, nb_epoch=10, batch_size=64,verbose=0)
    y_true,y_pred = get_Ytrue_Ypred(model,x_val,y_val)
    avg_acc.append(accuracy_score(y_true,y_pred))
    avg_f1.append(f1_score(y_true,y_pred,average='macro'))      
    print classification_report(y_true,y_pred)


print classification_report(y_true,y_pred)
print 'Average f1-score = ', np.mean(np.array(avg_f1))

             precision    recall  f1-score   support

       -1.0       0.85      0.92      0.88       438
        0.0       0.23      0.12      0.16        57
        1.0       0.53      0.46      0.49        70

avg / total       0.75      0.78      0.76       565

             precision    recall  f1-score   support

       -1.0       0.64      0.85      0.73       313
        0.0       0.42      0.18      0.26       131
        1.0       0.54      0.41      0.47       121

avg / total       0.57      0.60      0.57       565

             precision    recall  f1-score   support

       -1.0       0.42      0.92      0.58       219
        0.0       0.63      0.06      0.10       215
        1.0       0.61      0.34      0.43       131

avg / total       0.55      0.45      0.36       565

             precision    recall  f1-score   support

       -1.0       0.28      0.97      0.43       137
        0.0       1.00      0.02      0.04       379
        1.0       0.11      0.16    